Import the image and get the mask and edges


In [1]:
import numpy as np
import cv2
img = cv2.imread("pool-table-test2.jpg")

#rezize image
image = cv2.resize(img, (700, 600)) 

hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Defining lower and upper bound HSV values 
lower = np.array([50, 100, 100]) 
upper = np.array([70, 255, 255]) 

# Defining mask for detecting color 
mask = cv2.inRange(hsv, lower, upper) 

edges = cv2.Canny(image, 100, 200)

cv2.imshow("image", image)
cv2.imshow("Mask", mask) 
cv2.imshow("Edges", edges)


cv2.waitKey(0)

-1

In [2]:
import cv2
import numpy as np

# Load your image
img = cv2.imread("pool-table-test2.jpg")

image = cv2.resize(img, (500, 500)) 

# Create a callback function to record mouse clicks
def mouse_callback(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONUP:
        # Record the (x, y) coordinates of the mouse click
        print(f"Coordinates: ({x}, {y})")

# Create a window to display the image
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", mouse_callback)

while True:
    cv2.imshow("Image", image)
    key = cv2.waitKey(1) & 0xFF

    # Press 'q' to quit and close the window
    if key == ord('q'):
        break

cv2.destroyAllWindows()

Above is the code for finding the coordinates for the transformation


The transformation


In [3]:
import cv2
import numpy as np

# Load the image of the pool table
img = cv2.imread("pool-table-test2.jpg")
image = cv2.resize(img, (500, 500)) 


# coordinates of the corners of the pool table
image_points = np.array([[1, 215], [252, 114], [450, 139], [448, 305]], dtype=np.float32)

 
reference_points = np.array([[0, 0], [800, 0], [800, 600], [0, 600]], dtype=np.float32)

# Calculate the homography matrix
homography_matrix, _ = cv2.findHomography(image_points, reference_points, cv2.RANSAC, 5.0)

# Warp the image to get the top-down view
output_size = (800, 600)  # Define the size of the top-down view
warped_image = cv2.warpPerspective(image, homography_matrix, output_size)
warped_image_equalized = cv2.equalizeHist(cv2.cvtColor(warped_image, cv2.COLOR_BGR2GRAY))
warped_image_colormapped = cv2.applyColorMap(warped_image_equalized, cv2.COLORMAP_JET)

kernel = np.array([[-1,-1,-1],  [-1, 9,-1], [-1,-1,-1]])
sharpened_image = cv2.filter2D(warped_image, -1, kernel)


# Display all images 
cv2.imshow("Original Image", image)
cv2.imshow("Top-Down View", warped_image)
cv2.imshow("test", warped_image_equalized)
cv2.imshow("colormap", warped_image_colormapped)
cv2.imshow("Sharper image", sharpened_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [4]:
table= sharpened_image
edges=cv2.Canny(table,100,200)
cv2.imshow('Edges',edges)

Find the pool balls

In [5]:
import cv2
import numpy as np

# Load the image
image = sharpened_image



blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

# Convert the image to HSV color space 
hsv_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2HSV)

# Define the lower and upper bounds of the pool balls
lower_red = np.array([0, 100, 100])
upper_red = np.array([10, 255, 255])

lower_yellow = np.array([20, 100, 100])
upper_yellow = np.array([30, 255, 255])

lower_white = np.array([0,0,200])
upper_white = np.array([180,30,255])

# Create masks for pool balls
mask_red = cv2.inRange(hsv_image, lower_red, upper_red)
mask_yellow = cv2.inRange(hsv_image, lower_yellow, upper_yellow)
mask_white = cv2.inRange(hsv_image, lower_white, upper_white)

# use morpholohyEx to clean up the image
kernel = np.ones((5, 5), np.uint8)
mask_red = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel)
mask_yellow = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel)
mask_white = cv2.morphologyEx(mask_white, cv2.MORPH_CLOSE, kernel)

# Find contours in the masks
contours_red, _ = cv2.findContours(mask_red, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_yellow, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours_white,_ = cv2.findContours(mask_white, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create empty lists to store ball contours
ball_contours_red = []
ball_contours_yellow = []
ball_contours_white = []




# for loop to add red balls to list 
for contour in contours_red:
    area = cv2.contourArea(contour)
    min_area_threshold = 150  # Adjust as needed

    # Calculate the aspect ratio
    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h

    if area > min_area_threshold and 0.5 <= aspect_ratio <= 3:
        ball_contours_red.append(contour)

# for loop to add yellow balls to list 
for contour in contours_yellow:
    area = cv2.contourArea(contour)
    min_area_threshold = 150  # Adjust as needed

    if area > min_area_threshold:
        ball_contours_yellow.append(contour)

for contour in contours_white:
    area = cv2.contourArea(contour)
    min_area_threshold = 200

    x, y, w, h = cv2.boundingRect(contour)
    aspect_ratio = float(w) / h

    if area > min_area_threshold and 0.5 <= aspect_ratio <= 3:
        ball_contours_white.append(contour)



canvas = np.zeros_like(image)

# Draw the contours on the balls in their respective colours 
result_image = image.copy()
cv2.drawContours(result_image, ball_contours_red, -1, (0, 0, 255), 2)  # Red contours
cv2.drawContours(result_image, ball_contours_yellow, -1, (0, 255, 255), 2)  # Yellow contours
cv2.drawContours(result_image, ball_contours_white, -1, (255,255,255),2 )
cv2.imshow("Detected Balls", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Find the centre of the pool balls 

In [6]:
centers = []
centers_yellow =[]
center_white =[]

for contour in ball_contours_white:
    white_moments = cv2.moments(contour)

    if white_moments["m00"] != 0 :
        center_x = int(white_moments["m10"]/ white_moments["m00"])
        center_y = int(white_moments["m01"] / white_moments["m00"])
        center_white.append((center_x, center_y))

for contour in ball_contours_red:
     red_moments = cv2.moments(contour)

     if red_moments["m00"] != 0:
        center_x = int(red_moments["m10"]/ red_moments["m00"])
        center_y = int(red_moments["m01"] / red_moments["m00"])

        centers.append((center_x, center_y))

for contour in ball_contours_yellow:
    moments =cv2.moments(contour)

    if moments["m00"] != 0:
        center_x = int(moments["m10"]/ moments["m00"])
        center_y = int(moments["m01"] / moments["m00"])

        centers_yellow.append((center_x, center_y))

for center in centers:
    cv2.circle(result_image,center, 5, (0, 255 , 255),-1)

for center in centers_yellow:
    cv2.circle(result_image,center, 5, (0,0,255),-1) 

for center in center_white:
    cv2.circle(result_image,center, 5, (0, 0, 5), -1)       

print("centers red",centers)
print("Centers yellow",centers_yellow)
print("center white", center_white)
cv2.imshow("centers of balls",result_image)   
cv2.waitKey(0)
cv2.destroyAllWindows         

centers red [(628, 323), (694, 273), (68, 121)]
Centers yellow [(249, 382), (143, 378), (671, 260), (180, 252), (512, 96), (720, 75)]
center white [(657, 365)]


<function destroyAllWindows>

Draw more rounded pool balls using their centre as a reference point.


In [7]:
canvas = np.zeros_like(image)
# Draw red balls
for center in centers:
    cv2.circle(canvas, tuple(map(int, center)), 20, (0, 0, 255), -1)  # Red circle with radius 5

# Draw yellow balls
for center_yellow in centers_yellow:
    cv2.circle(canvas, tuple(map(int, center_yellow)), 20, (0, 255, 255), -1)  # Yellow circle with radius 5
#show canvas
cv2.imshow("Pool balls on canvas", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

Get mask of the table to create a 2d background for the balls


In [8]:
import cv2
import numpy as np

# Assume you have the warped image


# Convert the image to grayscale


# Use Canny edge detection to find edges in the image
edges_warped = cv2.Canny(blurred_image, 50, 150)

# Use HoughLinesP to detect lines in the image
lines = cv2.HoughLinesP(edges_warped, 1, np.pi / 180, threshold=10, minLineLength=100, maxLineGap=5)

# Draw the detected lines on a copy of the original image
lines_image = sharpened_image.copy()

if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(lines_image, (x1, y1), (x2, y2), (0, 255, 0), 2)

# Display the original image with detected lines
cv2.imshow("Lines on Warped Image", lines_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Find Pockets and pool table

In [9]:
import cv2
import numpy as np

# Load the sharpened image
image = sharpened_image

# Convert the image to grayscale
gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


blurred_image = cv2.GaussianBlur(gray_image, (5, 5), 0)

# Use the Canny edge detector
edges = cv2.Canny(blurred_image, 50, 150)

# Find contours in the edge-detected image
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create an empty list to store pocket contours
pocket_contours = []

# Iterate through the detected contours and filter out shapes resembling pockets
for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.02 * perimeter, True)

    # Assume pockets are rectangular or circular
    if len(approx) in [3, 4] or len(approx) > 6:
        pocket_contours.append(contour)

# Draw the detected pockets on the original image
result_image = image.copy()
cv2.drawContours(result_image, pocket_contours, -1, (0, 255, 0), 2)

# Display the result
cv2.imshow("Detected Pockets", result_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


Draw pool balls onto a canvas 

In [10]:
canvas = np.zeros_like(image)
#draw red balls
ball_contours_red.append(cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True))
cv2.drawContours(canvas, ball_contours_red,-1, (0,0,255),cv2.FILLED)
#draw yellow balls
ball_contours_yellow.append(cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True))
cv2.drawContours(canvas, ball_contours_yellow, -1, (0,255,255),cv2.FILLED)

#show canvas
cv2.imshow("Pool balls on canvas", canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()



In [11]:
import cv2
import numpy as np

# Load the image
image = sharpened_image

canvas1 = np.zeros_like(image)

# Convert the image to HSV color space
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the lower and upper bounds for the green color in HSV
lower_green = np.array([40, 40, 40])
upper_green = np.array([80, 255, 255])

# Create a binary mask for the specified green color range
mask_green = cv2.inRange(hsv_image, lower_green, upper_green)

# Apply morphological operations to clean up the mask (optional)
kernel = np.ones((5, 5), np.uint8)
mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)

# Find contours in the mask
contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the contour with the largest area (assuming it is the pool table)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a mask for the largest green region
    mask_largest_green = np.zeros_like(mask_green)
    cv2.drawContours(mask_largest_green, [largest_contour], -1, 255, thickness=cv2.FILLED)

    # Apply the mask to the original image to get the whole green region
    table_image = cv2.bitwise_and(image, image, mask=mask_largest_green)

    # Optional: Draw the largest green contour on the original image
    result_image = image.copy()
    cv2.drawContours(canvas1, [largest_contour], -1, (0, 255, 0), 2)  # green contour

    # Display the results
    cv2.imshow("Original Image", image)
    cv2.imshow("Detected Green Contour", canvas1)
    cv2.imshow("Whole Green Table", table_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No green contour found.")


In [12]:
canvas2 = np.zeros_like(image)
# Draw red balls
for center in centers:
    cv2.circle(canvas2, tuple(map(int, center)), 20, (0, 0, 255), -1)  # Red circle with radius 5

# Draw yellow balls
for center_yellow in centers_yellow:
    cv2.circle(canvas2, tuple(map(int, center_yellow)), 20, (0, 255, 255), -1)  # Yellow circle with radius 5
#show canvas2
cv2.imshow("Pool balls on canvas2", canvas2)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [13]:
canvas3 = np.zeros_like(image)

for center in centers:
    cv2.circle(canvas3, tuple(map(int, center)), 20, (0, 0, 255), -1)  # Red circle with radius 5

# Draw yellow balls
for center_yellow in centers_yellow:
    cv2.circle(canvas3, tuple(map(int, center_yellow)), 20, (0, 255, 255), -1)  # Yellow circle with radius 5

for center_white in center_white:
    cv2.circle(canvas, tuple(map(int, center_white)), 20, (0, 255, 255), -1)    

hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

# Define the lower and upper bounds for the green color in HSV
lower_green = np.array([40, 40, 40])
upper_green = np.array([80, 255, 255])

# Create a binary mask for the specified green color range
mask_green = cv2.inRange(hsv_image, lower_green, upper_green)

# Apply morphological operations to clean up the mask (optional)
kernel = np.ones((5, 5), np.uint8)
mask_green = cv2.morphologyEx(mask_green, cv2.MORPH_CLOSE, kernel)

# Find contours in the mask
contours, _ = cv2.findContours(mask_green, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the contour with the largest area (assuming it is the pool table)
if contours:
    largest_contour = max(contours, key=cv2.contourArea)

    # Create a mask for the largest green region
    mask_largest_green = np.zeros_like(mask_green)
    cv2.drawContours(mask_largest_green, [largest_contour], -1, 255, thickness=cv2.FILLED)

    # Apply the mask to the original image to get the whole green region
    table_image = cv2.bitwise_and(image, image, mask=mask_largest_green)

    # Optional: Draw the largest green contour on the original image
    result_image = image.copy()
    cv2.drawContours(canvas3, [largest_contour], -1, (0, 255, 0), 2)  # green contour

    cv2.imshow("pool table", canvas3)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("No green contour found.")


    



Animation for collison of pool balls using pygame

In [14]:
import pygame
import numpy as np

# Initialize Pygame
pygame.init()

# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Pool Ball Animation')

# Set up colors
POOL_TABLE_GREEN = (0, 100, 0)  # Dark green for the pool table
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BORDER_COLOR = (139, 69, 19)  # Dark brown for the table border
POCKET_COLOR = (0, 0, 0)  # Black for the pockets

# Ball class
class Ball:
    def __init__(self, position, velocity, color, static_friction_coefficient, rolling_friction_coefficient):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.color = color
        self.static_friction_coefficient = static_friction_coefficient  # Static friction coefficient
        self.rolling_friction_coefficient = rolling_friction_coefficient  # Rolling friction coefficient

    def update(self):
        self.velocity *= (1 - self.static_friction_coefficient)  # Apply static friction
        self.velocity -= self.velocity * self.rolling_friction_coefficient  # Apply rolling friction
        self.position += self.velocity

# Initialize arrays for ball positions
centers_yellow = [(249, 382), (143, 378), (671, 260), (180, 252), (512, 96), (720, 75)]
centers_red = [(628, 323), (694, 273), (68, 121)]
centers_white = [(657, 365)]

# Create balls based on the arrays
static_friction_coefficient = 0.002  # Adjust this value for static friction
rolling_friction_coefficient = 0.001  # Adjust this value for rolling friction
balls = [Ball(position=center, velocity=np.array([0.0, 0.0]), color=YELLOW, static_friction_coefficient=static_friction_coefficient, rolling_friction_coefficient=rolling_friction_coefficient) for center in centers_yellow]
balls += [Ball(position=center, velocity=np.array([0.0, 0.0]), color=RED, static_friction_coefficient=static_friction_coefficient, rolling_friction_coefficient=rolling_friction_coefficient) for center in centers_red]
balls += [Ball(position=center, velocity=np.array([2.0, 1.0]), color=WHITE, static_friction_coefficient=static_friction_coefficient, rolling_friction_coefficient=rolling_friction_coefficient) for center in centers_white]
# Set the initial velocity for the white ball
balls[-1].velocity = np.array([2.0, 1.0])  # Initial velocity for the white ball

# Main game loop
clock = pygame.time.Clock()
running = True
border_thickness = 20  # Thickness of the table border
pocket_radius = 32 #avg pocket size = ball radius * 1.6
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Update game state and draw
    screen.fill(POOL_TABLE_GREEN)  # Use green for the background

    # Draw pool table borders
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, width, border_thickness))  # Top border
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, border_thickness, height))  # Left border
    pygame.draw.rect(screen, BORDER_COLOR, (0, height - border_thickness, width, border_thickness))  # Bottom border
    pygame.draw.rect(screen, BORDER_COLOR, (width - border_thickness, 0, border_thickness, height))  # Right border

    # Draw pockets
    # Corners
    pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, height - border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, height - border_thickness), pocket_radius)
    # Sides
    pygame.draw.circle(screen, POCKET_COLOR, (width // 2, 2.5), pocket_radius) #top pocket
    pygame.draw.circle(screen, POCKET_COLOR, (width // 2, height - (border_thickness/8)), pocket_radius) #bottom middle pocket
   
    # Update ball positions
    for ball in balls:
        ball.update()

    # Check for ball-ball collisions and handle them
    for i in range(len(balls)):
        for j in range(i + 1, len(balls)):
            distance = np.linalg.norm(balls[i].position - balls[j].position)
            if distance <= 40:  # The sum of the radii is 20 * 2 = 40
                # Calculate the collision response
                normal = (balls[j].position - balls[i].position) / distance
                relative_velocity = balls[j].velocity - balls[i].velocity
                collision_speed = np.dot(relative_velocity, normal)
                if collision_speed < 0:
                    impulse = 2 * collision_speed / (1 + 1)  # 
                    balls[i].velocity += impulse * normal
                    balls[j].velocity -= impulse * normal

    # Check for ball-border collisions and handle them
    for ball in balls:
        if ball.velocity[0] != 0:
            if ball.position[0] < 20 or ball.position[0] > width - 20:
                ball.velocity[0] *= -1
        if ball.velocity[1] != 0:
            if ball.position[1] < 20 or ball.position[1] > height - 20:
                ball.velocity[1] *= -1

    # Draw balls
    for ball in balls:
        pygame.draw.circle(screen, ball.color, (int(ball.position[0]), int(ball.position[1])), 20)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


<frozen importlib._bootstrap>:241: RuntimeWarning: Your system is avx2 capable but pygame was not built with support for it. The performance of some of your blits could be adversely affected. Consider enabling compile time detection with environment variables like PYGAME_DETECT_AVX2=1 if you are compiling without cross compilation.


pygame 2.5.2 (SDL 2.28.3, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


Updated version with arrow for path selection


In [15]:
import pygame
import numpy as np
import math

# Initialize Pygame
pygame.init()

# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Pool Ball Animation')

# Set up colors
POOL_TABLE_GREEN = (0, 100, 0)  # Dark green for the pool table
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BORDER_COLOR = (139, 69, 19)  # Dark brown for the table border
POCKET_COLOR = (0, 0, 0)  # Black for the pockets
ARROW_COLOR = (255, 255, 255)  # White for the arrow

# Ball class
class Ball:
    def __init__(self, position, velocity, color, static_friction_coefficient, rolling_friction_coefficient):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.color = color
        self.static_friction_coefficient = static_friction_coefficient  # Static friction coefficient
        self.rolling_friction_coefficient = rolling_friction_coefficient  # Rolling friction coefficient
        self.radius = 20  # Radius of the ball for drawing

    def update(self):
        if np.linalg.norm(self.velocity) > 0.05:  # If velocity is not negligible
            self.velocity *= (1 - self.static_friction_coefficient)  # Apply static friction
            self.velocity -= self.velocity * self.rolling_friction_coefficient  # Apply rolling friction
        else:
            self.velocity = np.array([0.0, 0.0])  # Stop the ball if velocity is very low
        self.position += self.velocity

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.position.astype(int), self.radius)

# Initialize arrays for ball positions and set initial velocity of the white ball to zero
# Initialize arrays for ball positions
centers_yellow = [(249, 382), (143, 378), (671, 260), (180, 252), (512, 96), (720, 75)]
centers_red = [(628, 323), (694, 273), (68, 121)]
centers_white = [(657, 365)]

# Create balls based on the arrays
     
centers_white = [(657, 365)]
white_ball = Ball(centers_white[0], [0.0, 0.0], WHITE, 0.002, 0.001)
balls = [white_ball]

# Function to draw the arrow from the white ball towards the mouse position
def draw_arrow(screen, start_pos, end_pos, color):
    pygame.draw.line(screen, color, start_pos, end_pos, 2)  # Draw the line
    rotation = math.atan2(start_pos[1]-end_pos[1], end_pos[0]-start_pos[0])
    pygame.draw.polygon(screen, color, ((end_pos[0] + 10 * math.sin(rotation), end_pos[1] + 10 * math.cos(rotation)), 
                                         (end_pos[0] + 10 * math.sin(rotation - math.pi / 6), end_pos[1] + 10 * math.cos(rotation - math.pi / 6)), 
                                         (end_pos[0] + 10 * math.sin(rotation + math.pi / 6), end_pos[1] + 10 * math.cos(rotation + math.pi / 6))))

# Main game loop
clock = pygame.time.Clock()
running = True
arrow_drawn = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not arrow_drawn:
            mouse_pos = pygame.mouse.get_pos()
            direction = np.array(mouse_pos) - white_ball.position
            norm_direction = np.linalg.norm(direction)
            if norm_direction > 0:
                direction = direction / norm_direction
            white_ball.velocity = direction * 5  # Set velocity based on direction and speed
            arrow_drawn = True  # Prevents further arrow drawing after click

    # Update game state and draw
    screen.fill(POOL_TABLE_GREEN)  # Use green for the background

    # Pool table borders and pockets drawing code remains the same...
     # Draw pool table borders
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, width, border_thickness))  # Top border
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, border_thickness, height))  # Left border
    pygame.draw.rect(screen, BORDER_COLOR, (0, height - border_thickness, width, border_thickness))  # Bottom border
    pygame.draw.rect(screen, BORDER_COLOR, (width - border_thickness, 0, border_thickness, height))  # Right border

    # Draw pockets
    # Corners
    pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, height - border_thickness), pocket_radius)
    pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, height - border_thickness), pocket_radius)
    # Sides
    pygame.draw.circle(screen, POCKET_COLOR, (width // 2, 2.5), pocket_radius) #top pocket
    pygame.draw.circle(screen, POCKET_COLOR, (width // 2, height - (border_thickness/8)), pocket_radius) #bottom middle pocket

    # Check for ball-ball collisions and handle them
    for i in range(len(balls)):
        for j in range(i + 1, len(balls)):
            distance = np.linalg.norm(balls[i].position - balls[j].position)
            if distance <= 40:  # The sum of the radii is 20 * 2 = 40
                # Calculate the collision response
                normal = (balls[j].position - balls[i].position) / distance
                relative_velocity = balls[j].velocity - balls[i].velocity
                collision_speed = np.dot(relative_velocity, normal)
                if collision_speed < 0:
                    impulse = 2 * collision_speed / (1 + 1)  # 
                    balls[i].velocity += impulse * normal
                    balls[j].velocity -= impulse * normal

    # Check for ball-border collisions and handle them
    for ball in balls:
        if ball.velocity[0] != 0:
            if ball.position[0] < 20 or ball.position[0] > width - 20:
                ball.velocity[0] *= -1
        if ball.velocity[1] != 0:
            if ball.position[1] < 20 or ball.position[1] > height - 20:
                ball.velocity[1] *= -1

    # Update and draw balls
    for ball in balls:
        ball.update()
        ball.draw(screen)

    # Draw the arrow from the white ball towards the mouse position if the arrow is not drawn yet
    if not arrow_drawn:
        mouse_pos = pygame.mouse.get_pos()
        draw_arrow(screen, white_ball.position, mouse_pos, ARROW_COLOR)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


with arrow and all balls and collisons 


In [16]:
import pygame
import numpy as np
import math

# Initialize Pygame
pygame.init()



# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Pool Ball Animation')

# Set up colors
POOL_TABLE_GREEN = (0, 100, 0)  # Dark green for the pool table
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BORDER_COLOR = (139, 69, 19)  # Dark brown for the table border
POCKET_COLOR = (0, 0, 0)  # Black for the pockets
ARROW_COLOR = (255, 255, 255)  # White for the arrow



# Ball class
class Ball:
    def __init__(self, position, velocity, color, static_friction_coefficient, rolling_friction_coefficient):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.color = color
        self.static_friction_coefficient = static_friction_coefficient  # Static friction coefficient
        self.rolling_friction_coefficient = rolling_friction_coefficient  # Rolling friction coefficient
        self.radius = 20  # Radius of the ball for drawing

    def update(self):
        if np.linalg.norm(self.velocity) > 0.05:  # If velocity is not negligible
            self.velocity *= (1 - self.static_friction_coefficient)  # Apply static friction
            self.velocity -= self.velocity * self.rolling_friction_coefficient  # Apply rolling friction
        else:
            self.velocity = np.array([0.0, 0.0])  # Stop the ball if velocity is very low
        self.position += self.velocity

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.position.astype(int), self.radius)

# Initialize ball positions
centers_yellow = [(249, 382), (143, 378), (671, 260), (180, 252), (512, 96), (720, 75)]
centers_red = [(628, 323), (694, 273), (68, 121)]
centers_white = [(657, 365)]

# Create balls
static_friction_coefficient = 0.002
rolling_friction_coefficient = 0.003
balls = [Ball(center, [0.0, 0.0], color, static_friction_coefficient, rolling_friction_coefficient) 
         for center, color in zip(centers_yellow, [YELLOW]*len(centers_yellow))]
balls += [Ball(center, [0.0, 0.0], color, static_friction_coefficient, rolling_friction_coefficient) 
          for center, color in zip(centers_red, [RED]*len(centers_red))]
balls += [Ball(centers_white[0], [0.0, 0.0], WHITE, static_friction_coefficient, rolling_friction_coefficient)]

white_ball = balls[-1]  # Reference to the white ball for drawing the arrow

# Function to draw the arrow
def draw_arrow(screen, start_pos, end_pos, color):
    pygame.draw.line(screen, color, start_pos, end_pos, 2)  # Draw the line
    rotation = math.atan2(start_pos[1]-end_pos[1], end_pos[0]-start_pos[0])
    pygame.draw.polygon(screen, color, ((end_pos[0] + 10 * math.sin(rotation), end_pos[1] + 10 * math.cos(rotation)), 
                                         (end_pos[0] + 10 * math.sin(rotation - math.pi / 6), end_pos[1] + 10 * math.cos(rotation - math.pi / 6)), 
                                         (end_pos[0] + 10 * math.sin(rotation + math.pi / 6), end_pos[1] + 10 * math.cos(rotation + math.pi / 6))))

#fuction to check for potted balls
def check_ball_pocket_collisions(balls, pocket_positions, pocket_threshold):
    global width, height, border_thickness
    # Iterate through balls in reverse to allow removing without affecting the iteration
    for i in range(len(balls) - 1, -1, -1):
        ball = balls[i]
        for pocket_pos in pocket_positions:
            distance = np.linalg.norm(ball.position - np.array(pocket_pos))
            # If the ball is close enough to the pocket, consider it "pocketed"
            if distance < pocket_threshold:
                # Remove the ball from the list, effectively making it "disappear"
                del balls[i]
                break  # Break out of the loop after removing the ball

# Define pocket positions based on your pocket drawing logic
pocket_positions = [
    (border_thickness, border_thickness),
    (width - border_thickness, border_thickness),
    (border_thickness, height - border_thickness),
    (width - border_thickness, height - border_thickness),
    (width // 2, 2.5),
    (width // 2, height - (border_thickness / 8))
]
pocket_threshold = pocket_radius
# Main game loop
clock = pygame.time.Clock()
running = True
arrow_drawn = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not arrow_drawn:
            mouse_pos = pygame.mouse.get_pos()
            direction = np.array(mouse_pos) - white_ball.position
            norm_direction = np.linalg.norm(direction)
            if norm_direction > 0:
                direction = direction / norm_direction
            white_ball.velocity = direction * 5  # Set velocity based on direction and speed
            arrow_drawn = True  # Prevents further arrow drawing after click

    # Update game state and draw
    screen.fill(POOL_TABLE_GREEN)

    # Draw pool table borders
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, width, border_thickness))  # Top border
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, border_thickness, height))  # Left border
    pygame.draw.rect(screen, BORDER_COLOR, (0, height - border_thickness, width, border_thickness))  # Bottom border
    pygame.draw.rect(screen, BORDER_COLOR, (width - border_thickness, 0, border_thickness, height))  # Right border

    # Draw pockets
    # Corners
    pocket1= pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, border_thickness), pocket_radius)
    pocket2= pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, border_thickness), pocket_radius)
    pocket3= pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, height - border_thickness), pocket_radius)
    pocket4= pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, height - border_thickness), pocket_radius)
    # Sides
    pocket5= pygame.draw.circle(screen, POCKET_COLOR, (width // 2, 2.5), pocket_radius) #top pocket
    pocket6= pygame.draw.circle(screen, POCKET_COLOR, (width // 2, height - (border_thickness/8)), pocket_radius) #bottom middle pocket

    # Check for ball-ball collisions and handle them
    for i in range(len(balls)):
        for j in range(i + 1, len(balls)):
            distance = np.linalg.norm(balls[i].position - balls[j].position)
            if distance <= 40:  # The sum of the radii is 20 * 2 = 40
                # Calculate the collision response
                normal = (balls[j].position - balls[i].position) / distance
                relative_velocity = balls[j].velocity - balls[i].velocity
                collision_speed = np.dot(relative_velocity, normal)
                if collision_speed < 0:
                    impulse = 2 * collision_speed / (1 + 1)  # 
                    balls[i].velocity += impulse * normal
                    balls[j].velocity -= impulse * normal

    # Check for ball-border collisions and handle them
    for ball in balls:
    # For the X direction
     if ball.velocity[0] != 0:
        # Check collision with the left border
        if ball.position[0] - ball.radius < border_thickness:
            ball.velocity[0] *= -1
            ball.position[0] = border_thickness + ball.radius  # Reposition ball to avoid getting stuck in the border
        # Check collision with the right border
        elif ball.position[0] + ball.radius > width - border_thickness:
            ball.velocity[0] *= -1
            ball.position[0] = width - border_thickness - ball.radius  # Reposition ball to avoid getting stuck in the border

    # For the Y direction
     if ball.velocity[1] != 0:
        # Check collision with the top border
        if ball.position[1] - ball.radius < border_thickness:
            ball.velocity[1] *= -1
            ball.position[1] = border_thickness + ball.radius  # Reposition ball to avoid getting stuck in the border
        # Check collision with the bottom border
        elif ball.position[1] + ball.radius > height - border_thickness:
            ball.velocity[1] *= -1
            ball.position[1] = height - border_thickness - ball.radius  # Reposition ball to avoid getting stuck in the border

     
         #check for potted balls and make them vanish
    check_ball_pocket_collisions(balls, pocket_positions, pocket_threshold)
    
    # Update and draw balls
    for ball in balls:
        ball.update()
        ball.draw(screen)

    # Draw the arrow from the white ball towards the mouse position if the arrow is not drawn yet
    if not arrow_drawn:
        mouse_pos = pygame.mouse.get_pos()
        draw_arrow(screen, white_ball.position, mouse_pos, ARROW_COLOR)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()


In [ ]:
import pygame
import numpy as np
import math

# Initialize Pygame
pygame.init()



# Set up display
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption('Pool Ball Animation')

# Set up colors
POOL_TABLE_GREEN = (0, 100, 0)  # Dark green for the pool table
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BORDER_COLOR = (139, 69, 19)  # Dark brown for the table border
POCKET_COLOR = (0, 0, 0)  # Black for the pockets
ARROW_COLOR = (255, 255, 255)  # White for the arrow



# Ball class
class Ball:
    def __init__(self, position, velocity, color, static_friction_coefficient, rolling_friction_coefficient):
        self.position = np.array(position, dtype=float)
        self.velocity = np.array(velocity, dtype=float)
        self.color = color
        self.static_friction_coefficient = static_friction_coefficient  # Static friction coefficient
        self.rolling_friction_coefficient = rolling_friction_coefficient  # Rolling friction coefficient
        self.radius = 20  # Radius of the ball for drawing

    def update(self):
        if np.linalg.norm(self.velocity) > 0.05:  # If velocity is not negligible
            self.velocity *= (1 - self.static_friction_coefficient)  # Apply static friction
            self.velocity -= self.velocity * self.rolling_friction_coefficient  # Apply rolling friction
        else:
            self.velocity = np.array([0.0, 0.0])  # Stop the ball if velocity is very low
        self.position += self.velocity

    def draw(self, screen):
        pygame.draw.circle(screen, self.color, self.position.astype(int), self.radius)

# Initialize ball positions
centers_yellow = [(249, 382), (143, 378), (671, 260), (180, 252), (512, 96), (720, 75)]
centers_red = [(628, 323), (694, 273), (68, 121)]
centers_white = [(657, 365)]

# Create balls
static_friction_coefficient = 0.002
rolling_friction_coefficient = 0.003
balls = [Ball(center, [0.0, 0.0], color, static_friction_coefficient, rolling_friction_coefficient) 
         for center, color in zip(centers_yellow, [YELLOW]*len(centers_yellow))]
balls += [Ball(center, [0.0, 0.0], color, static_friction_coefficient, rolling_friction_coefficient) 
          for center, color in zip(centers_red, [RED]*len(centers_red))]
balls += [Ball(centers_white[0], [0.0, 0.0], WHITE, static_friction_coefficient, rolling_friction_coefficient)]

white_ball = balls[-1]  # Reference to the white ball for drawing the arrow

# Function to draw the arrow
def draw_arrow(screen, start_pos, end_pos, color):
    pygame.draw.line(screen, color, start_pos, end_pos, 2)  # Draw the line
    rotation = math.atan2(start_pos[1]-end_pos[1], end_pos[0]-start_pos[0])
    pygame.draw.polygon(screen, color, ((end_pos[0] + 10 * math.sin(rotation), end_pos[1] + 10 * math.cos(rotation)), 
                                         (end_pos[0] + 10 * math.sin(rotation - math.pi / 6), end_pos[1] + 10 * math.cos(rotation - math.pi / 6)), 
                                         (end_pos[0] + 10 * math.sin(rotation + math.pi / 6), end_pos[1] + 10 * math.cos(rotation + math.pi / 6))))

#fuction to check for potted balls
def check_ball_pocket_collisions(balls, pocket_positions, pocket_threshold):
    global width, height, border_thickness
    # Iterate through balls in reverse to allow removing without affecting the iteration
    for i in range(len(balls) - 1, -1, -1):
        ball = balls[i]
        for pocket_pos in pocket_positions:
            distance = np.linalg.norm(ball.position - np.array(pocket_pos))
            # If the ball is close enough to the pocket, consider it "pocketed"
            if distance < pocket_threshold:
                # Remove the ball from the list, effectively making it "disappear"
                del balls[i]
                break  # Break out of the loop after removing the ball

# Define pocket positions based on your pocket drawing logic
pocket_positions = [
    (border_thickness, border_thickness),
    (width - border_thickness, border_thickness),
    (border_thickness, height - border_thickness),
    (width - border_thickness, height - border_thickness),
    (width // 2, 2.5),
    (width // 2, height - (border_thickness / 8))
]
pocket_threshold = pocket_radius
# Main game loop
clock = pygame.time.Clock()
running = True
arrow_drawn = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN and not arrow_drawn:
            mouse_pos = pygame.mouse.get_pos()
            direction = np.array(mouse_pos) - white_ball.position
            norm_direction = np.linalg.norm(direction)
            if norm_direction > 0:
                direction = direction / norm_direction
            white_ball.velocity = direction * 5  # Set velocity based on direction and speed
            arrow_drawn = True  # Prevents further arrow drawing after click

    # Update game state and draw
    screen.fill(POOL_TABLE_GREEN)

    # Draw pool table borders
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, width, border_thickness))  # Top border
    pygame.draw.rect(screen, BORDER_COLOR, (0, 0, border_thickness, height))  # Left border
    pygame.draw.rect(screen, BORDER_COLOR, (0, height - border_thickness, width, border_thickness))  # Bottom border
    pygame.draw.rect(screen, BORDER_COLOR, (width - border_thickness, 0, border_thickness, height))  # Right border

    # Draw pockets
    # Corners
    pocket1= pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, border_thickness), pocket_radius)
    pocket2= pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, border_thickness), pocket_radius)
    pocket3= pygame.draw.circle(screen, POCKET_COLOR, (border_thickness, height - border_thickness), pocket_radius)
    pocket4= pygame.draw.circle(screen, POCKET_COLOR, (width - border_thickness, height - border_thickness), pocket_radius)
    # Sides
    pocket5= pygame.draw.circle(screen, POCKET_COLOR, (width // 2, 2.5), pocket_radius) #top pocket
    pocket6= pygame.draw.circle(screen, POCKET_COLOR, (width // 2, height - (border_thickness/8)), pocket_radius) #bottom middle pocket

    # Check for ball-ball collisions and handle them
    for i in range(len(balls)):
        for j in range(i + 1, len(balls)):
            distance = np.linalg.norm(balls[i].position - balls[j].position)
            if distance <= 40:  # The sum of the radii is 20 * 2 = 40
                # Calculate the collision response
                normal = (balls[j].position - balls[i].position) / distance
                relative_velocity = balls[j].velocity - balls[i].velocity
                collision_speed = np.dot(relative_velocity, normal)
                if collision_speed < 0:
                    impulse = 2 * collision_speed / (1 + 1)  # 
                    balls[i].velocity += impulse * normal
                    balls[j].velocity -= impulse * normal

    # Check for ball-border collisions and handle them
    for ball in balls:
    # For the X direction
     if ball.velocity[0] != 0:
        # Check collision with the left border
        if ball.position[0] - ball.radius < border_thickness:
            ball.velocity[0] *= -1
            ball.position[0] = border_thickness + ball.radius  # Reposition ball to avoid getting stuck in the border
        # Check collision with the right border
        elif ball.position[0] + ball.radius > width - border_thickness:
            ball.velocity[0] *= -1
            ball.position[0] = width - border_thickness - ball.radius  # Reposition ball to avoid getting stuck in the border

    # For the Y direction
     if ball.velocity[1] != 0:
        # Check collision with the top border
        if ball.position[1] - ball.radius < border_thickness:
            ball.velocity[1] *= -1
            ball.position[1] = border_thickness + ball.radius  # Reposition ball to avoid getting stuck in the border
        # Check collision with the bottom border
        elif ball.position[1] + ball.radius > height - border_thickness:
            ball.velocity[1] *= -1
            ball.position[1] = height - border_thickness - ball.radius  # Reposition ball to avoid getting stuck in the border

     
         #check for potted balls and make them vanish
    check_ball_pocket_collisions(balls, pocket_positions, pocket_threshold)
    
    # Update and draw balls
    for ball in balls:
        ball.update()
        ball.draw(screen)

    # Draw the arrow from the white ball towards the mouse position if the arrow is not drawn yet
    if not arrow_drawn:
        mouse_pos = pygame.mouse.get_pos()
        draw_arrow(screen, white_ball.position, mouse_pos, ARROW_COLOR)

    pygame.display.flip()
    clock.tick(60)

pygame.quit()
